In [1]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
import xgboost as xgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation, LSTM, Conv1D, MaxPooling1D, Flatten
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error, r2_score
from math import sqrt
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.callbacks import ModelCheckpoint
import keras
from keras.models import load_model
from tensorflow.keras.layers import InputLayer
import xgboost

In [2]:
train_data = pd.read_csv('train_data.csv')
test_data = pd.read_csv('test_data.csv')
columns = train_data.drop(['PDB ID','G_experiment'], axis = 1).columns
X_train = train_data.drop(['PDB ID','G_experiment'], axis = 1)
Y_train = train_data['G_experiment']
validation_size = int( 0.15 * len(X_train) )
X_train = X_train[:-validation_size]
Y_train = Y_train[:-validation_size]
X_val = X_train[-validation_size:]
Y_val = Y_train[-validation_size:]
X_test  = test_data.drop(['PDB ID','G_experiment','RR_Predict_G', 'RFR_Predict_G', 'SVR_Predict_G',
       'XGBR_Predict_G'], axis = 1)
Y_test  = test_data['G_experiment']
scaler = MinMaxScaler()
s_scaler = StandardScaler()
X_train_m = scaler.fit_transform(X_train)
X_val_m = scaler.fit_transform(X_val)
X_test_m  = scaler.transform(X_test)
X_train_s = s_scaler.fit_transform(X_train)
X_val_s = s_scaler.fit_transform(X_val)
X_test_s  = s_scaler.transform(X_test)


X_train_m_2, Y_train_m_2 = np.array(X_train_m), np.array(Y_train)
X_val_m_2, Y_val_m_2 = np.array(X_val_m), np.array(Y_val)
X_test_m_2, Y_test_m_2 = np.array(X_test_m), np.array(Y_test)
X_train_s_2, Y_train_s_2 = np.array(X_train_m), np.array(Y_train)
X_val_s_2, Y_val_s_2 = np.array(X_val_m), np.array(Y_val)
X_test_s_2, Y_test_s_2 = np.array(X_test_m), np.array(Y_test)

X_train_m_2 = X_train_m_2.reshape((X_train_m_2.shape[0], X_train_m_2.shape[1], 1))
X_val_m_2 = X_val_m_2.reshape((X_val_m_2.shape[0], X_val_m_2.shape[1], 1)) 
X_test_m_2 = X_test_m_2.reshape((X_test_m_2.shape[0], X_test_m_2.shape[1], 1))
X_train_s_2 = X_train_s_2.reshape((X_train_s_2.shape[0], X_train_s_2.shape[1], 1)) 
X_val_s_2 = X_val_s_2.reshape((X_val_s_2.shape[0], X_val_s_2.shape[1], 1)) 
X_test_s_2 = X_test_s_2.reshape((X_test_s_2.shape[0], X_test_s_2.shape[1], 1))

## Decision Tree

In [21]:
criterions = ['squared_error', 'friedman_mse' ]
max_depths = [5, 10, 50 , 100]
min_samples_splits = [ 2, 4, 10]
min_leaf_samples = [1, 3, 5]
max_leaf_nodes = [ 3, 5, 10, 15]
dt_best_params_m = {}
dt_best_params_s = {}
best_mse_m = float('inf')
best_mse_s = float('inf')
for criterion in criterions:
    for max_depth in max_depths:
        for min_samples_split in min_samples_splits:
            for min_samples_leaf in min_leaf_samples:
                for max_leaf_node in max_leaf_nodes:                
                    dt_m = DecisionTreeRegressor(criterion = criterion, max_depth = max_depth, min_samples_split = min_samples_split, min_samples_leaf = min_samples_leaf, max_leaf_nodes = max_leaf_node)
                    dt_s = DecisionTreeRegressor(criterion = criterion, max_depth = max_depth, min_samples_split = min_samples_split, min_samples_leaf = min_samples_leaf, max_leaf_nodes = max_leaf_node)
                    dt_m.fit(X_train_m,Y_train)
                    dt_s.fit(X_train_s,Y_train)
                    pred_m = dt_m.predict(X_val_m)
                    pred_s = dt_s.predict(X_val_s)
                    mse_m = mean_squared_error(Y_val,pred_m)
                    mse_s = mean_squared_error(Y_val,pred_s)
                    if mse_m < best_mse_m:
                        best_mse_m = mse_m
                        dt_best_params_m['criterion'] = criterion
                        dt_best_params_m['max_depth'] = max_depth
                        dt_best_params_m['min_samples_split'] = min_samples_split
                        dt_best_params_m['min_samples_leaf'] = min_samples_leaf
                        dt_best_params_m['max_leaf_nodes'] = max_leaf_node
                    if mse_s < best_mse_s:
                        best_mse_s = mse_s
                        dt_best_params_s['criterion'] = criterion
                        dt_best_params_s['max_depth'] = max_depth
                        dt_best_params_s['min_samples_split'] = min_samples_split
                        dt_best_params_s['min_samples_leaf'] = min_samples_leaf
                        dt_best_params_s['max_leaf_nodes'] = max_leaf_node
print(best_mse_m, dt_best_params_m)
print(best_mse_s, dt_best_params_s)

4.921906492799171 {'criterion': 'squared_error', 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_leaf_nodes': 10}
4.5712766625992165 {'criterion': 'squared_error', 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_leaf_nodes': 15}


## Random Forest

In [18]:
n_estimatorss = [5, 10, 50, 100]
criterions = ['squared_error', 'friedman_mse' ]
max_depths = [5, 10, 50 , 100]
min_samples_splits = [ 2, 4, 10]
min_leaf_samples = [1, 3, 5]
max_leaf_nodes = [ 3, 5, 10, 15]
rf_best_params_m = {}
rf_best_params_s = {}
best_mse_m = float('inf')
best_mse_s = float('inf')
for n_estimators in n_estimatorss:
    for criterion in criterions:
        for max_depth in max_depths:
            for min_samples_split in min_samples_splits:
                for min_samples_leaf in min_leaf_samples:
                    for max_leaf_node in max_leaf_nodes:                
                        dt_m = RandomForestRegressor(n_estimators, criterion = criterion, max_depth = max_depth, min_samples_split = min_samples_split, min_samples_leaf = min_samples_leaf, max_leaf_nodes = max_leaf_node)
                        dt_s = RandomForestRegressor(n_estimators, criterion = criterion, max_depth = max_depth, min_samples_split = min_samples_split, min_samples_leaf = min_samples_leaf, max_leaf_nodes = max_leaf_node)
                        dt_m.fit(X_train_m,Y_train)
                        dt_s.fit(X_train_s,Y_train)
                        pred_m = dt_m.predict(X_val_m)
                        pred_s = dt_s.predict(X_val_s)
                        mse_m = mean_squared_error(Y_val,pred_m)
                        mse_s = mean_squared_error(Y_val,pred_s)
                        if mse_m < best_mse_m:
                            best_mse_m = mse_m
                            rf_best_params_m['n_estimators'] = n_estimators
                            rf_best_params_m['criterion'] = criterion
                            rf_best_params_m['max_depth'] = max_depth
                            rf_best_params_m['min_samples_split'] = min_samples_split
                            rf_best_params_m['min_samples_leaf'] = min_samples_leaf
                            rf_best_params_m['max_leaf_nodes'] = max_leaf_node
                        if mse_s < best_mse_s:
                            best_mse_s = mse_s
                            rf_best_params_s['n_estimators'] = n_estimators
                            rf_best_params_s['criterion'] = criterion
                            rf_best_params_s['max_depth'] = max_depth
                            rf_best_params_s['min_samples_split'] = min_samples_split
                            rf_best_params_s['min_samples_leaf'] = min_samples_leaf
                            rf_best_params_s['max_leaf_nodes'] = max_leaf_node
print(best_mse_m, rf_best_params_m)
print(best_mse_s, rf_best_params_s)

KeyboardInterrupt: 

## SVM

In [24]:
kernels = ['rbf', 'poly', 'sigmoid']
degrees = [2, 3, 4]
Cs = [1, 5, 10, 50]
gammas = [0.05, 0.01, 0.005, 0.001]
dt_best_params_m = {}
dt_best_params_s = {}
best_mse_m = float('inf')
best_mse_s = float('inf')

for kernel in kernels:
    for C in Cs:
        for gamma in gammas:
            if kernel != 'poly':
                svm_m = SVR(kernel = kernel, C = C, gamma = gamma)
                svm_s = SVR(kernel = kernel, C = C, gamma = gamma)
                svm_m.fit(X_train_m,Y_train)
                svm_s.fit(X_train_s,Y_train)
                pred_m = svm_m.predict(X_val_m)
                pred_s = svm_s.predict(X_val_s)
                mse_m = mean_squared_error(Y_val,pred_m)
                mse_s = mean_squared_error(Y_val,pred_s)
                if mse_m < best_mse_m:
                    best_mse_m = mse_m
                    dt_best_params_m['kernel'] = kernel
                    dt_best_params_m['C'] = C
                    dt_best_params_m['gamma'] = gamma
                if mse_s < best_mse_s:
                    best_mse_s = mse_s
                    dt_best_params_s['kernel'] = kernel
                    dt_best_params_s['C'] = C
                    dt_best_params_s['gamma'] = gamma
            else:
                for degree in degrees:
                    svm_m = SVR(kernel = kernel, C = C, gamma = gamma, degree = degree)
                    svm_s = SVR(kernel = kernel, C = C, gamma = gamma, degree = degree)
                    svm_m.fit(X_train_m,Y_train)
                    svm_s.fit(X_train_s,Y_train)
                    pred_m = svm_m.predict(X_val_m)
                    pred_s = svm_s.predict(X_val_s)
                    mse_m = mean_squared_error(Y_val,pred_m)
                    mse_s = mean_squared_error(Y_val,pred_s)
                    if mse_m < best_mse_m:
                        best_mse_m = mse_m
                        dt_best_params_m['kernel'] = kernel
                        dt_best_params_m['C'] = C
                        dt_best_params_m['gamma'] = gamma
                        dt_best_params_m['degree'] = degree
                    if mse_s < best_mse_s:
                        best_mse_s = mse_s
                        dt_best_params_s['kernel'] = kernel
                        dt_best_params_s['C'] = C
                        dt_best_params_s['gamma'] = gamma
                        dt_best_params_s['degree'] = degree
print(best_mse_m, dt_best_params_m)
print(best_mse_s, dt_best_params_s)

5.3848711834145035 {'kernel': 'sigmoid', 'C': 50, 'gamma': 0.001, 'degree': 2}
3.190967255666647 {'kernel': 'rbf', 'C': 50, 'gamma': 0.01}


## K-NN

In [25]:
n_neighborss = [2, 3, 5, 7]
dt_best_params_m = {}
dt_best_params_s = {}
best_mse_m = float('inf')
best_mse_s = float('inf')
for n_neighbors in n_neighborss:
    knn_m = KNeighborsRegressor(n_neighbors)
    knn_s = KNeighborsRegressor(n_neighbors)
    knn_m.fit(X_train_m,Y_train)
    knn_s.fit(X_train_s,Y_train)
    pred_m = knn_m.predict(X_val_m)
    pred_s = knn_s.predict(X_val_s)
    mse_m = mean_squared_error(Y_val,pred_m)
    mse_s = mean_squared_error(Y_val,pred_s)
    if mse_m < best_mse_m:
        best_mse_m = mse_m
        dt_best_params_m['n_neighbors'] = n_neighbors
    if mse_s < best_mse_s:
        best_mse_s = mse_s
        dt_best_params_s['n_neighbors'] = n_neighbors
print(best_mse_m, dt_best_params_m)
print(best_mse_s, dt_best_params_s)

5.997087101148183 {'n_neighbors': 7}
1.5890184141795267 {'n_neighbors': 2}


## XGBoost

In [6]:
depths = [10, 20, 50, 100, 600]
lrs = [0.1, 0.05, 0.01, 0.001, 0.0005, 0.0001]
estims = [10, 50, 100, 1000, 2000]
dt_best_params_m = {}
dt_best_params_s = {}
best_mse_m = float('inf')
best_mse_s = float('inf')
for depth in depths:
    for lr in lrs:
        for estim in estims:
            knn_m = xgboost.XGBRegressor(n_estimators=estim, max_depth=depth, learning_rate = lr)
            knn_s = xgboost.XGBRegressor(n_estimators=estim, max_depth=depth, learning_rate = lr)
            knn_m.fit(X_train_m,Y_train)
            knn_s.fit(X_train_s,Y_train)
            pred_m = knn_m.predict(X_val_m)
            pred_s = knn_s.predict(X_val_s)
            mse_m = mean_squared_error(Y_val,pred_m)
            mse_s = mean_squared_error(Y_val,pred_s)
            if mse_m < best_mse_m:
                best_mse_m = mse_m
                dt_best_params_m['n_estimators'] = estim
                dt_best_params_m['max_depth'] = depth
                dt_best_params_m['learning_rate'] = lr
            if mse_s < best_mse_s:
                best_mse_s = mse_s
                dt_best_params_s['n_estimators'] = estim
                dt_best_params_s['max_depth'] = depth
                dt_best_params_s['learning_rate'] = lr
print(best_mse_m, dt_best_params_m)
print(best_mse_s, dt_best_params_s)


7.514997000306236 {'n_estimators': 1000, 'max_depth': 10, 'learning_rate': 0.05}
2.2959680168376075 {'n_estimators': 1000, 'max_depth': 10, 'learning_rate': 0.1}


## MLP

In [9]:
def create_mlp(neurons, input_size, final_activation, learning_rate):
    model_mlp = Sequential()
    model_mlp.add(Dense(neurons[0], activation='relu', input_dim = input_size))
    for neuron in neurons[1:]:
        model_mlp.add(Dense(neuron, activation='relu'))
    model_mlp.add(Dense(1, activation = final_activation))
    model_mlp.compile(optimizer= tf.keras.optimizers.Adam(learning_rate= learning_rate), loss = 'mean_squared_error')
    return model_mlp

neurons = [[50, 30, 10], [80, 50, 30, 10], [100, 80, 50], [ 30, 10]]
final_activations = ['relu', 'linear']
learning_rates = [0.1, 0.05, 0.01, 0.001, 0.0005]
input_size = len(X_train_m[0])
batch_size = 16
total_epochs = 100
dt_best_params_m = {}
dt_best_params_s = {}
best_mse_m = float('inf')
best_mse_s = float('inf')

for neuron in neurons:
    for final_activation in final_activations:
        for learning_rate in learning_rates:
            knn_m = create_mlp(neuron, input_size, final_activation, learning_rate)
            knn_s = create_mlp(neuron, input_size, final_activation, learning_rate)
            history_m = knn_m.fit(X_train_m,Y_train, batch_size= batch_size, epochs=total_epochs, verbose = 0)
            history_s = knn_s.fit(X_train_s,Y_train, batch_size= batch_size, epochs=total_epochs, verbose = 0)
            pred_m = knn_m.predict(X_val_m)
            pred_s = knn_s.predict(X_val_s)
            mse_m = mean_squared_error(Y_val,pred_m)
            mse_s = mean_squared_error(Y_val,pred_s)
            if mse_m < best_mse_m:
                best_mse_m = mse_m
                dt_best_params_m['neuron'] = neuron
                dt_best_params_m['final_activation'] = final_activation
                dt_best_params_m['learning_rate'] = learning_rate
            if mse_s < best_mse_s:
                best_mse_s = mse_s
                dt_best_params_s['neuron'] = neuron
                dt_best_params_s['final_activation'] = final_activation
                dt_best_params_s['learning_rate'] = learning_rate
print(best_mse_m, dt_best_params_m)
print(best_mse_s, dt_best_params_s)

18/18 [==============================] - 0s 1ms/step
5.741033988356952 {'n_estimators': 1000, 'max_depth': 10, 'learning_rate': 0.0005, 'neuron': [80, 50, 30, 10], 'final_activation': 'linear'}
1.6095435423152222 {'n_estimators': 1000, 'max_depth': 10, 'learning_rate': 0.001, 'neuron': [100, 80, 50], 'final_activation': 'linear'}


## LSTM

In [13]:
def create_lstm(neurons, input_size, final_activation, learning_rate):
    model_mlp = Sequential()
    model_mlp.add(LSTM(neurons[0], activation='relu', return_sequences=True, input_shape=(input_size[0], input_size[1])))
    for neuron in neurons[1:-1]:
        model_mlp.add(LSTM(neuron, activation='relu', return_sequences=True))
    model_mlp.add(LSTM(5, activation = 'relu'))
    model_mlp.add(Dense(1, activation = final_activation))
    model_mlp.compile(optimizer= tf.keras.optimizers.Adam(learning_rate= learning_rate), loss = 'mean_squared_error')
    return model_mlp

neurons = [[50, 30, 10], [80, 50, 30, 10], [100, 80, 50], [ 30, 10, 5]]
final_activations = [ 'linear']
learning_rates = [ 0.0001, 0.001, 0.0005]
input_size = [X_train_m_2.shape[1], X_train_m_2.shape[2]]
batch_size = 128
total_epochs = 100

dt_best_params_m = {}
dt_best_params_s = {}
best_mse_m = float('inf')
best_mse_s = float('inf')

for neuron in neurons:
    for final_activation in final_activations:
        for learning_rate in learning_rates:
            knn_m = create_lstm(neuron, input_size, final_activation, learning_rate)
            knn_s = create_lstm(neuron, input_size, final_activation, learning_rate)
            history_m = knn_m.fit(X_train_m_2,Y_train_m_2, batch_size= batch_size, epochs=total_epochs, verbose = 0)
            history_s = knn_s.fit(X_train_s_2,Y_train_s_2, batch_size= batch_size, epochs=total_epochs, verbose = 0)
            pred_m = knn_m.predict(X_val_m_2)
            pred_s = knn_s.predict(X_val_s_2)
            try:
                mse_m = mean_squared_error(Y_val_m_2,pred_m)
                mse_s = mean_squared_error(Y_val_s_2,pred_s)
            except:
                continue
            if mse_m < best_mse_m:
                best_mse_m = mse_m
                dt_best_params_m['neuron'] = neuron
                dt_best_params_m['final_activation'] = final_activation
                dt_best_params_m['learning_rate'] = learning_rate
            if mse_s < best_mse_s:
                best_mse_s = mse_s
                dt_best_params_s['neuron'] = neuron
                dt_best_params_s['final_activation'] = final_activation
                dt_best_params_s['learning_rate'] = learning_rate
print(best_mse_m, dt_best_params_m)
print(best_mse_s, dt_best_params_s)

18/18 [==============================] - 1s 14ms/step
5.426601541081951 {'neuron': [80, 50, 30, 10], 'final_activation': 'linear', 'learning_rate': 0.001}
5.333393428273475 {'neuron': [80, 50, 30, 10], 'final_activation': 'linear', 'learning_rate': 0.0005}


## CNN

In [14]:
def create_cnn(filters, kernel, input_size, final_activation, learning_rate):
    model_mlp = Sequential()
    model_mlp.add(Conv1D(filters=filters, kernel_size=kernel, activation='relu', input_shape=(input_size[0], input_size[1])))
    model_mlp.add(MaxPooling1D(pool_size=2))
    model_mlp.add(Flatten())
    model_mlp.add(Dense(1, activation = final_activation))
    model_mlp.compile(optimizer= tf.keras.optimizers.Adam(learning_rate= learning_rate), loss = 'mean_squared_error')
    return model_mlp

filterss = [16, 32, 64]
kernels = [3,5,7]
final_activations = ['linear']
learning_rates = [ 0.01, 0.001, 0.0005]
input_size = [X_train_m_2.shape[1], X_train_m_2.shape[2]]
batch_size = 16
total_epochs = 100
                     
dt_best_params_m = {}
dt_best_params_s = {}
best_mse_m = float('inf')
best_mse_s = float('inf')

for filters in filterss:
    for kernel in kernels:
        for learning_rate in learning_rates:
            knn_m = create_cnn(filters, kernel, input_size, final_activation, learning_rate)
            knn_s = create_cnn(filters, kernel, input_size, final_activation, learning_rate)
            history_m = knn_m.fit(X_train_m,Y_train, batch_size= batch_size, epochs=total_epochs, verbose = 0)
            history_s = knn_s.fit(X_train_s,Y_train, batch_size= batch_size, epochs=total_epochs, verbose = 0)
            pred_m = knn_m.predict(X_val_m)
            pred_s = knn_s.predict(X_val_s)
            try:
                mse_m = mean_squared_error(Y_val,pred_m)
                mse_s = mean_squared_error(Y_val,pred_s)
            except:
                continue
            if mse_m < best_mse_m:
                best_mse_m = mse_m
                dt_best_params_m['filters'] = filters
                dt_best_params_m['kernel'] = kernel
                dt_best_params_m['learning_rate'] = learning_rate
            if mse_s < best_mse_s:
                best_mse_s = mse_s
                dt_best_params_s['filters'] = filters
                dt_best_params_s['kernel'] = kernel
                dt_best_params_s['learning_rate'] = learning_rate
print(best_mse_m, dt_best_params_m)
print(best_mse_s, dt_best_params_s)

18/18 [==============================] - 0s 2ms/step
5.315335101203674 {'filters': 16, 'kernel': 3, 'learning_rate': 0.0005}
3.605669539725345 {'filters': 64, 'kernel': 7, 'learning_rate': 0.001}


## LSTM-CNN

In [15]:
def create_lstm_cnn(neurons, filters, kernel, input_size, final_activation, learning_rate):
    model_mlp = Sequential()
    model_mlp.add(LSTM(neurons[0], activation='relu', return_sequences=True, input_shape=(input_size[0], input_size[1])))
    for neuron in neurons[1:]:
        model_mlp.add(LSTM(neuron, activation='relu', return_sequences=True))
    model_mlp.add(Conv1D(filters=filters, kernel_size=kernel, activation='relu'))
    model_mlp.add(MaxPooling1D(pool_size=2))
    model_mlp.add(Flatten())
    model_mlp.add(Dense(10, activation = 'relu'))
    model_mlp.add(Dense(1, activation = final_activation))
    model_mlp.compile(optimizer= tf.keras.optimizers.Adam(learning_rate= learning_rate), loss = 'mean_squared_error')
    return model_mlp

neurons = [[50, 30, 10], [80, 50, 30, 10], [100, 80, 50], [ 30, 10, 5]]
filterss = [16, 32, 64]
kernels = [3,5,7]
final_activations = ['linear']
learning_rates = [ 0.01, 0.001, 0.0005]
input_size = [X_train_m_2.shape[1], X_train_m_2.shape[2]]
batch_size = 128
total_epochs = 100
                     
dt_best_params_m = {}
dt_best_params_s = {}
best_mse_m = float('inf')
best_mse_s = float('inf')

for neuron in neurons:
    for filters in filterss:
        for kernel in kernels:
            for learning_rate in learning_rates:
                knn_m = create_lstm_cnn(neuron, filters, kernel, input_size, final_activation, learning_rate)
                knn_s = create_lstm_cnn(neuron, filters, kernel, input_size, final_activation, learning_rate)
                history_m = knn_m.fit(X_train_m,Y_train, batch_size= batch_size, epochs=total_epochs, verbose = 0)
                history_s = knn_s.fit(X_train_s,Y_train, batch_size= batch_size, epochs=total_epochs, verbose = 0)
                pred_m = knn_m.predict(X_val_m)
                pred_s = knn_s.predict(X_val_s)
                try:
                    mse_m = mean_squared_error(Y_val,pred_m)
                    mse_s = mean_squared_error(Y_val,pred_s)
                except:
                    continue
                if mse_m < best_mse_m:
                    best_mse_m = mse_m
                    dt_best_params_m['neuron'] = neuron
                    dt_best_params_m['filters'] = filters
                    dt_best_params_m['learning_rate'] = learning_rate
                if mse_s < best_mse_s:
                    best_mse_s = mse_s
                    dt_best_params_s['neuron'] = neuron
                    dt_best_params_s['filters'] = filters
                    dt_best_params_s['kernel'] = kernel
                    dt_best_params_s['learning_rate'] = learning_rate
print(best_mse_m, dt_best_params_m)
print(best_mse_s, dt_best_params_s)

18/18 [==============================] - 1s 50ms/step


KeyboardInterrupt: 

In [16]:
print(best_mse_m, dt_best_params_m)
print(best_mse_s, dt_best_params_s)

5.105683237353674 {'neuron': [50, 30, 10], 'filters': 32, 'kernel': 7, 'learning_rate': 0.001}
4.21329445855553 {'neuron': [50, 30, 10], 'filters': 64, 'kernel': 7, 'learning_rate': 0.001}
